In [1]:
#import some libraries
import nltk
from pprint import pprint
import csv
from typing import List
import re
from collections import Counter
import numpy as np
import pandas as pd
from IPython.display import clear_output
import pickle
from sklearn.externals import joblib

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble, cluster
from sklearn.decomposition import PCA
from sklearn.externals import joblib
from nltk.tokenize import word_tokenize,sent_tokenize

from nltk.corpus import stopwords # library 
nltk.download('stopwords')
all_stopwords = set(stopwords.words('english')) # set the language 
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nagem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# #remove stop-words
# from nltk.corpus import stopwords # library 
# nltk.download('stopwords')
# all_stopwords = set(stopwords.words('english')) # set the language 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nagem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#load datset
df = pd.read_csv('C:/Users/nagem/OneDrive/Documents/MSBA 2019/570/clash_of_clans_570/clash-of-clans.csv')
reviews_data = pd.DataFrame({'review': df.Content, 'sentiment': df.Rating})
print(reviews_data.head(2))

# reading review data with panda frames 
reviews_data.describe()


review  sentiment
0               💜💜💜          5
1  Pls make it th13          3


,sentiment
count,50001.000000
mean,4.612628
std,1.026489
min,1.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


The dataset was loaded and 2 columns were kept, the one with reviews and the sentiment score which was on a scale of 1 to 5.

In [3]:
# encode non-anscii to anscii and then decode string
reviews_data['review'] = reviews_data['review'].apply(lambda x: str(x).encode('ascii','ignore').decode())

The reviews had some non-anscii characters which had to be encoded and decoded.

In [4]:
# The sentiments are either 'positive' or 'negative' and are evenly distributed. Lets preprocess the text using the simple tokenizer we built in last class. We call it preprocess_text now.
def preprocess_text(text):
    pattern1 = re.compile("<br /><br />|\.")
    lines = re.split(pattern1, text)
    tokens = []
    for line in lines:
        tokens += line.split(" ")

    # lowercase and remove any non-alphanumeric characters from tokens for normalize
    normalized_tokens = [token for token in tokens if re.search(r"^[A-Za-z]\w*$", token) is not None]
    return  " ".join([
            token
            for token in normalized_tokens
            if token and token not in all_stopwords and len(tokens) > 1
        ])
    

  
custom_review = "I hated the film. It was a disaster. Poor direction, bad acting."
custom_review_tokens = preprocess_text(custom_review)
print(custom_review_tokens)

I hated film It disaster Poor bad acting


The reviews were cleaned and split into words. Normalization removed punctuation and stopwords. 

In [5]:
#apply preprocessing to review data
reviews_data['review'] = reviews_data['review'].apply(preprocess_text)

In [6]:
lower_count_thr = 1000 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens

vectorizer=TfidfVectorizer(min_df=lower_count_thr,max_df=upper_count_thr,binary=False,ngram_range=(1,1))

Initialy the TfidVectorizer was tried with a lower threshold of 1000 and an upper count of 5000. The ngram_range was set to (1,1).

In [7]:
tfidf_vectors = vectorizer.fit_transform(reviews_data['review'] )

In [8]:
feature_names = vectorizer.get_feature_names()
dense = tfidf_vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [9]:
print(len(df))
print(df[10:30])

50001
    awesome  best       coc  ever  great   it     like  love  much  play  \
10      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
11      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
12      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
13      0.0   0.0  0.000000   0.0    0.0  0.0  0.64249   0.0   0.0   0.0   
14      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
15      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
16      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
17      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
18      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
19      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   0.0   0.0   0.0   
20      0.0   0.0  0.000000   0.0    1.0  0.0  0.00000   0.0   0.0   0.0   
21      0.0   0.0  0.000000   0.0    0.0  0.0  0.00000   1.0   0.0   0.0   
22    

In [10]:
num_clust = 3
km = KMeans(n_clusters = num_clust, max_iter=100, n_init=1)

In [11]:
km.fit(tfidf_vectors)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=3, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [12]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clust):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 love
 much
 coc
 play
 best
 supercell
 like
 it
 awesome
 great
Cluster 1:
 best
 like
 very
 it
 play
 time
 ever
 great
 super
 coc
Cluster 2:
 this
 best
 love
 play
 world
 awesome
 like
 ever
 time
 much


The resulting clusters were very positive and somewhat similar sounding. 

In [13]:
#Count vectorizer
def run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter=100, n_init=1): 
    vectorizer=TfidfVectorizer(min_df=lower_count_thr,max_df=upper_count_thr,binary=False,ngram_range=(n_gram_min,n_gram_max))

    tfidf_vectors = vectorizer.fit_transform(reviews)

    feature_names = vectorizer.get_feature_names()
    dense = tfidf_vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)

    # print('Length of denselist:\n', len(df))
    # print('#10-30 from denselist:\n',df[10:30])

    km = KMeans(n_clusters = num_clust, max_iter=max_iter, n_init=n_init)

    km.fit(tfidf_vectors)

    print("Top terms per cluster:")
    order_centroids = km.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(num_clust):
        print("\nCluster %d:" % i),
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind]),
        print

A function was created to run multiple Kmeans with one cell that had adjustable parameters. Different variations were tried to see which would result in useful clusters. 

In [15]:
lower_count_thr = 500 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 1
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Top terms per cluster:

Cluster 0:
 super
 this
 love
 play
 like
 very
 world
 time
 it
 games

Cluster 1:
 best
 very
 this
 play
 time
 ever
 awesome
 coc
 world
 please

Cluster 2:
 like
 much
 play
 this
 coc
 love
 really
 it
 games
 best

Cluster 3:
 love
 much
 coc
 this
 best
 clash
 play
 playing
 clans
 supercell

Cluster 4:
 it
 great
 time
 best
 play
 love
 fun
 world
 awesome
 really


Since and overwhelming percentage of reviews were 5 star and had similar words, the lower and upper threshold were lowered to captured more unique words. 

In [16]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 1000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 1
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Top terms per cluster:

Cluster 0:
 keep
 work
 new
 playing
 updates
 going
 update
 get
 games
 fun

Cluster 1:
 amazing
 games
 fun
 its
 the
 cool
 addictive
 update
 clash
 played

Cluster 2:
 graphics
 gameplay
 strategy
 amazing
 high
 addictive
 loved
 the
 cool
 troops

Cluster 3:
 hai
 pubg
 update
 download
 clans
 clash
 guys
 superb
 hall
 town

Cluster 4:
 upgrade
 lot
 takes
 long
 take
 fun
 get
 troops
 things
 town


In [17]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 1
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Top terms per cluster:

Cluster 0:
 it
 best
 world
 play
 interesting
 like
 love
 awesome
 great
 amazing

Cluster 1:
 love
 best
 very
 like
 super
 ever
 great
 play
 time
 world

Cluster 2:
 hai
 pubg
 update
 coc
 very
 download
 love
 clans
 supercell
 clash

Cluster 3:
 this
 awesome
 best
 love
 world
 amazing
 like
 play
 ever
 interesting

Cluster 4:
 coc
 love
 best
 pubg
 like
 play
 world
 playing
 better
 games


Experiments with different number of ngrams were tested to see if more information could be interpreted from the clusters.

In [18]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 2
reviews = reviews_data['review']

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews)    

Top terms per cluster:

Cluster 0:
 please
 update
 give
 game please
 make
 add
 fix
 please add
 supercell
 please fix

Cluster 1:
 nice game
 love
 good game
 like
 this
 love game
 it
 super
 awesome
 play

Cluster 2:
 best
 best game
 ever
 game ever
 world
 game world
 played
 this best
 ever played
 this

Cluster 3:
 very
 very nice
 very good
 nice game
 good game
 addictive
 interesting
 bad
 interesting game
 much

Cluster 4:
 clash
 clans
 clash clans
 clash clan
 clan
 love clash
 love
 best
 play
 like


In [19]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 5000 # frequent/common tokens
num_clust = 5
n_gram_min = 1
n_gram_max = 2
reviews = reviews_data['review']
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

Top terms per cluster:

Cluster 0:
 super
 super game
 game super
 very
 it
 this
 world
 games
 like
 game world

Cluster 1:
 nice game
 very nice
 very
 so
 game like
 its
 like
 world
 game world
 game nice

Cluster 2:
 please
 update
 give
 game please
 make
 add
 fix
 please add
 supercell
 please fix

Cluster 3:
 good game
 best
 best game
 like
 this
 it
 very
 play
 awesome
 great

Cluster 4:
 love
 love game
 game love
 much
 game much
 this
 nice game
 this game
 play
 good game


In [20]:
lower_count_thr = 100 # rare words/tokens
upper_count_thr = 800 # frequent/common tokens
num_clust = 5
n_gram_min = 3
n_gram_max = 3
reviews = reviews_data['review']
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

Top terms per cluster:

Cluster 0:
 it good game
 world best game
 love game much
 best strategy game
 this game good
 this best game
 the best game
 this good game
 like game much
 it best game

Cluster 1:
 very nice game
 game love game
 game best game
 best game world
 like game much
 best strategy game
 game ever played
 it best game
 it good game
 it nice game

Cluster 2:
 very good game
 game love game
 game best game
 this good game
 world best game
 this best game
 best game world
 this game best
 the best game
 very nice game

Cluster 3:
 best game world
 this best game
 it best game
 game best game
 the best game
 this game best
 world best game
 love game much
 like game much
 this good game

Cluster 4:
 game ever played
 best strategy game
 this best game
 the best game
 it best game
 game best game
 this game best
 game love game
 like game much
 world best game


In [21]:
rating1 = reviews_data[reviews_data['sentiment']==1]['review']
print('Length rating1: ', len(rating1))

rating2 = reviews_data[reviews_data['sentiment']==2]['review']
print('Length rating2: ', len(rating2))

rating3 = reviews_data[reviews_data['sentiment']==3]['review']
print('Length rating3: ', len(rating3))

rating4 = reviews_data[reviews_data['sentiment']==4]['review']
print('Length rating4: ', len(rating4))

rating5 = reviews_data[reviews_data['sentiment']==5]['review']
print('Length rating5: ', len(rating5))

Length rating1:  2877
Length rating2:  583
Length rating3:  1398
Length rating4:  3316
Length rating5:  41827


The reviews were then separated into groups based on rating and clusters made of each group.

## 1 star rating

In [22]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.9) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 2
n_gram_max = 4
reviews = rating1
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

50 45001
Top terms per cluster:

Cluster 0:
 this game
 supercell id
 good game
 play game
 game ever
 bad game
 worst game

Cluster 1:
 worst game
 game ever
 play game
 this game
 bad game
 supercell id
 good game

Cluster 2:
 bad game
 game ever
 play game
 this game
 good game
 worst game
 supercell id


The one star rating had a large amount of negative words and some positive. Of the three clusters above, two were related to supercell sugesting this may be a factor in disatisfied users. 

In [23]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating1
max_iter = 200
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 2302
Top terms per cluster:

Cluster 0:
 friend cant login
 login stuck loading
 login stuck loading page
 cant login stuck
 cant login stuck loading
 stuck loading page
 friend cant login stuck
 zyada boring hai seriously
 game thank devs
 game thank devs ruining

Cluster 1:
 very bad game
 worst game ever
 very good game
 pubg better coc
 game ever played
 worst game world
 time wasting game
 it good game
 worst game ever played
 clan war league

Cluster 2:
 much cache recent update
 much cache recent
 taking much cache recent
 taking much cache
 cache recent update
 zyada boring hai seriously
 game th lev
 game th12 coming
 game thank devs
 game thank devs ruining


We hypothesized that the majority of reviews in any rating category would be farely generic so further lowered the threshold  in hopes of capturing more unique and descriptive words. When the cluster focused more on rare words and use ngram of (3,4), we began to see some interesting things come out. In the clusters above, the cluster 0 seems to have pretty generic negative reviews. For cluster 1, one of the commonly repeated thems is devs, suggesting the players blame the developers for the negative state of the game. Cluster 2 references the tropps and battles with the emphasis on how unpredictable of inconsistent they were. 

In [24]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.45) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 5
n_gram_min = 3
n_gram_max = 4
reviews = rating1
max_iter = 50
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

3 1295
Top terms per cluster:

Cluster 0:
 pubg better coc
 would give stars
 google play game
 it nice game
 it is so
 it good game
 it bad game
 is so bad
 id much upgraded face
 id much upgraded

Cluster 1:
 very bad game
 clan war league
 worst game world
 very good game
 it good game
 time wasting game
 remove supercell id
 this game bad
 bad game world
 the worst game

Cluster 2:
 super cell id
 using supercell id
 please please please
 login google play
 google play game
 back google play
 it nice game
 it is so
 it good game
 it bad game

Cluster 3:
 clash clans game
 time killer game
 would give stars
 google play sign
 it nice game
 it is so
 it good game
 it bad game
 is so bad
 id much upgraded face

Cluster 4:
 worst game ever
 worst game ever played
 game ever played
 game ever play
 the worst game
 this game worst
 game worst game
 google play sign
 game love game
 it good game


A somewhat frequent occurance in the last few clusters was mention of google or google play. There maybe some aspect of google play that is causing users to have a bad experience. Another common occurance references "id much upgraded" or "id much upgraded face".

## 2 star rating

In [25]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.5) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating2
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

3 1438
Top terms per cluster:

Cluster 0:
 time searching opponent
 game ever played
 clan war league
 town hall level
 it good game

Cluster 1:
 town hall level
 time searching opponent
 it good game
 game ever played
 clan war league

Cluster 2:
 it good game
 town hall level
 time searching opponent
 game ever played
 clan war league


In [26]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating2
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 466
Top terms per cluster:

Cluster 0:
 consuming lot battery
 add flag pakistan
 many maintenance break
 my troops showing
 kisaan mastar kishan
 this game died
 coc dont work
 wow brilliant game
 like better pubg
 need good grapics

Cluster 1:
 acc lock hours
 zuerst bestimmte gebude bauen
 game things wrong
 game starting hve edton
 game starting someone
 game starting someone may
 game stoped loading
 game stoped loading plz
 game stops back
 game stops back home

Cluster 2:
 game third class activity
 worst game third class
 worst game third
 class activity work
 third class activity
 third class activity work
 game third class
 zuerst bestimmte gebude bauen
 game stops back home
 game stops back


Looking at the two star reviews, we see a some different topics occuring. In the clusters above, cluster 0 references money which may be related to the game requiring players to spend money in order to get better weapons. They are able to achieve it but it takes a very long time. cluster 2 references how long it takes to achieve nakakawala, again related to the time required to get certain things without spending money. 

## 3 star rating

In [27]:
lower_count_thr = round(len(reviews)*.001) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 3
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 466
Top terms per cluster:

Cluster 0:
 one alredy attached
 got star clan
 alredy attached base
 star clan war
 star clan war clear
 clan war clear
 clan war clear one
 war clear one alredy
 one alredy attached base
 got star clan war

Cluster 1:
 very good game
 best game ever
 very nice game
 world best game
 best strategic game
 nice game time
 good pubg best
 good game like
 time pass game
 good strategic game

Cluster 2:
 this good game
 zwischen truppe und fertig
 get behind the addition
 get enough loot upgrade
 get enough loot
 get easy clan games
 get easy clan
 get damaged heavily make
 get damaged heavily
 get clan war builder


At 3 stars we start to see more positive sentiment again. In the above clusters, cluster 0 has pretty positive sentiment. Cluster 1 seems to talk about difficulties getting enough loot. Cluster 2 appears to be related to users that felt the main had become stale. 

In [29]:
lower_count_thr = round(len(reviews)*.0001) # rare words/tokens
upper_count_thr = round(len(reviews)*.6) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 2
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

0 839
Top terms per cluster:

Cluster 0:
 very good game
 this good game
 best game ever
 very nice game
 world best game
 nice game time
 best strategic game
 good pubg best
 good game like
 time pass game

Cluster 1:
 please include flag pakistan
 include flag pakistan
 please include flag
 get behind the
 get enough loot
 get easy clan games
 get easy clan
 get damaged heavily make
 get damaged heavily
 get clan war builder


Another clustering of 3 start reviews had a class related to progress restart of deleted.

In [30]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.8) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 1118
Top terms per cluster:

Cluster 0:
 like update problum
 zwischen truppe und fertig
 get behind the addition
 get far takes
 get enough loot upgrade
 get enough loot
 get easy clan games
 get easy clan
 get damaged heavily make
 get damaged heavily

Cluster 1:
 very good game
 this good game
 best game ever
 very nice game
 world best game
 nice game time
 best strategic game
 good pubg best
 good game like
 time pass game

Cluster 2:
 for refreshing mind
 zwischen truppe und fertig
 get clan war
 get far takes
 get enough loot upgrade
 get enough loot
 get easy clan games
 get easy clan
 get damaged heavily make
 get damaged heavily

Cluster 3:
 updates time big
 give updates time big
 give updates time
 please give updates
 please give updates time
 get behind the
 get enough loot
 get easy clan games
 get easy clan
 get damaged heavily make


In cluster 3 there are references to when connection was very bad during Christmas. 

In [31]:
lower_count_thr = round(len(reviews)*.0015) # rare words/tokens
upper_count_thr = round(len(reviews)*.65) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating3
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

2 909
Top terms per cluster:

Cluster 0:
 very good game
 this good game
 like game much
 best game ever
 world best game
 it good game
 overall good game
 very nice game
 good pubg best
 good game time

Cluster 1:
 clan war league
 written maintaining break
 experience good lost account
 good game like
 good game but
 gmail account pls help
 gmail account pls
 give new update change
 give new update
 getting really bored waiting

Cluster 2:
 this game good
 game good time
 written maintaining break
 games take forever
 good game like
 good game but
 gmail account pls help
 gmail account pls
 give new update change
 give new update

Cluster 3:
 good game like
 its good game
 written maintaining break
 game would play
 good game but
 gmail account pls help
 gmail account pls
 give new update change
 give new update
 getting really bored waiting


In the clusters above, we again see gmail account mentioned. There may be issues that users are experiencing related to google or gmail accounts. 

## 4 star rating

In [32]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.4) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating4
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

1 559
Top terms per cluster:

Cluster 0:
 best game ever
 it good game
 very good game
 very nice game
 this good game
 good time pass
 its good game
 it best game
 nice game like
 nice game play

Cluster 1:
 great time killer
 zoom see base find
 game wait time
 game verry much
 game verry much please
 game very cool
 game very cool game
 game very very
 game very very much
 game wait bit

Cluster 2:
 my favourite game
 zoom see base find
 game wait time upgrade
 game verry much please
 game very cool
 game very cool game
 game very very
 game very very much
 game wait bit
 game wait bit much

Cluster 3:
 help get gems
 superb help get
 superb help get gems
 game world best
 game used people
 game used play
 game used play came
 game ver good
 game verry much
 game verry much please


With the 4 star ratings, we seem to have more positive sentiment than the previous ratings. Two new topics also appear. One refers to children and the other to dark or gold elixir.

In [33]:
## 5 star rating

In [110]:
lower_count_thr = round(len(reviews)*.0005) # rare words/tokens
upper_count_thr = round(len(reviews)*.3) # frequent/common tokens
print(lower_count_thr, upper_count_thr)
num_clust = 4
n_gram_min = 3
n_gram_max = 4
reviews = rating5
max_iter = 100
n_init = 5

run_kmeans(lower_count_thr, upper_count_thr, num_clust, n_gram_min, n_gram_max, reviews, max_iter, n_init)

2 995
Length of denselist:
 41827
#10-30 from denselist:
     a1 game world  able join clan  able play friends  able to understand  \
10            0.0             0.0                0.0                 0.0   
11            0.0             0.0                0.0                 0.0   
12            0.0             0.0                0.0                 0.0   
13            0.0             0.0                0.0                 0.0   
14            0.0             0.0                0.0                 0.0   
15            0.0             0.0                0.0                 0.0   
16            0.0             0.0                0.0                 0.0   
17            0.0             0.0                0.0                 0.0   
18            0.0             0.0                0.0                 0.0   
19            0.0             0.0                0.0                 0.0   
20            0.0             0.0                0.0                 0.0   
21            0.0             

The 5 star ratings were very positive in tone and did not provide much information on details about things that people liked or things that could be improved. 

The purpose of this research was in part to determine features of the game that players either liked or did not like so the developers could make improvements in those areas. 

* From one star ratings we see that players are likely unhappy about unpredictable troops and battle. 
* Two star reviews feel negatively about how long it takes to get further in the game without spending money. 
* For three star reviews the clusters suggest that players are having issue with their gmail or google play accounts. 
* Four star reviews reference dark or gold elixir it is seen with references to better and children. 
* The five star reviews did not seem to reference specific as the other ratings did. However, these were the majority of the reviews which suggests the game is well liked and negative things mentioned in lower star reviews do not greatly impact the majority of players badly. 